In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import json
from bs4 import BeautifulSoup
import numpy as np
import requests
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [2]:
with open('vetting_playlist.json', 'r') as file:
    data1 = json.load(file)

In [22]:
tracks_data = [item['track'] for item in data1['tracks']['items']]
artists_data = [artist['name'] for item in tracks_data for artist in item['artists']]
# Convert to DataFrame
dataF = pd.DataFrame(tracks_data)
d1 = pd.DataFrame(artists_data)
dataF# all data
d1#artist

,0
0,Lake Street Dive
1,Heartless Bastards
2,Vulfpeck
3,Alabama Shakes
4,Bahamas
...,...
150,Ezra Furman
151,Abby Sage
152,Alex Ebert
153,JosÃ© GonzÃ¡lez


In [23]:
dataF

,preview_url,available_markets,explicit,type,episode,track,album,artists,disc_number,track_number,duration_ms,external_ids,external_urls,href,id,name,popularity,uri,is_local
0,None,[],False,track,False,True,"{'available_markets': [], 'type': 'album', 'al...",[{'external_urls': {'spotify': 'https://open.s...,1,3,215879,{'isrc': 'USFMT1406103'},{'spotify': 'https://open.spotify.com/track/0N...,https://api.spotify.com/v1/tracks/0NSEZeWHkYCk...,0NSEZeWHkYCknk1nAVO1Fh,Better Than,0,spotify:track:0NSEZeWHkYCknk1nAVO1Fh,False
1,None,[],False,track,False,True,"{'available_markets': [], 'type': 'album', 'al...",[{'external_urls': {'spotify': 'https://open.s...,1,4,308066,{'isrc': 'USBQU1100085'},{'spotify': 'https://open.spotify.com/track/17...,https://api.spotify.com/v1/tracks/17ZnveSDBpG9...,17ZnveSDBpG9QtL7zLJNPy,Only For You,0,spotify:track:17ZnveSDBpG9QtL7zLJNPy,False
2,None,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",False,track,False,True,"{'available_markets': ['AR', 'AU', 'AT', 'BE',...",[{'external_urls': {'spotify': 'https://open.s...,1,2,181259,{'isrc': 'TCACH1515840'},{'spotify': 'https://open.spotify.com/track/1D...,https://api.spotify.com/v1/tracks/1DrlLvlYd1FI...,1DrlLvlYd1FIjNavRm6NdX,Back Pocket,61,spotify:track:1DrlLvlYd1FIjNavRm6NdX,False
3,None,[],False,track,False,True,"{'available_markets': [], 'type': 'album', 'al...",[{'external_urls': {'spotify': 'https://open.s...,1,3,257920,{'isrc': 'USATO1400852'},{'spotify': 'https://open.spotify.com/track/51...,https://api.spotify.com/v1/tracks/51ggEZWbFP5d...,51ggEZWbFP5dMdWjGCDYTk,Dunes,0,spotify:track:51ggEZWbFP5dMdWjGCDYTk,False
4,None,[],False,track,False,True,"{'available_markets': [], 'type': 'album', 'al...",[{'external_urls': {'spotify': 'https://open.s...,1,4,234973,{'isrc': 'USUG11400491'},{'spotify': 'https://open.spotify.com/track/40...,https://api.spotify.com/v1/tracks/40WXqoyOLAAu...,40WXqoyOLAAugGmJWZU1FY,All The Time,18,spotify:track:40WXqoyOLAAugGmJWZU1FY,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
141,None,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",False,track,False,True,"{'available_markets': ['AR', 'AU', 'AT', 'BE',...",[{'external_urls': {'spotify': 'https://open.s...,1,9,294966,{'isrc': 'GBBRP1651004'},{'spotify': 'https://open.spotify.com/track/3m...,https://api.spotify.com/v1/tracks/3mTD0EKHUaNU...,3mTD0EKHUaNU2Cp6PTJGpa,I Can Change,37,spotify:track:3mTD0EKHUaNU2Cp6PTJGpa,False
142,None,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",False,track,False,True,"{'available_markets': ['AR', 'AU', 'AT', 'BE',...",[{'external_urls': {'spotify': 'https://open.s...,1,1,187072,{'isrc': 'CAN112200678'},{'spotify': 'https://open.spotify.com/track/5v...,https://api.spotify.com/v1/tracks/5vbnQr1tWSSR...,5vbnQr1tWSSREmcIONIUm9,Backwards Directions,56,spotify:track:5vbnQr1tWSSREmcIONIUm9,False
143,None,[],False,track,False,True,"{'available_markets': [], 'type': 'album', 'al...",[{'external_urls': {'spotify': 'https://open.s...,1,3,260920,{'isrc': 'USVR91165603'},{'spotify': 'https://open.spotify.com/track/4d...,https://api.spotify.com/v1/tracks/4d0YWF2D8hXo...,4d0YWF2D8hXoyUs8LBkYHg,Truth,9,spotify:track:4d0YWF2D8hXoyUs8LBkYHg,False
144,None,"[CA, PR, US]",False,track,False,True,"{'available_markets': ['CA', 'PR', 'US'], 'typ...",[{'external_urls': {'spotify': 'https://open.s...,1,11,188080,{'isrc': 'SEVVX2100111'},{'spotify': 'https://open.spotify.com/track/2E...,https://api.spotify.com/v1/tracks/2ET7YTxeo4tn...,2ET7YTxeo4tn7BlUGs3t4Y,Line Of Fire,43,spotify:track:2ET7YTxeo4tn7BlUGs3t4Y,False


In [4]:
file_paths = [
    'mpd.slice.15000-15999.json',
    'mpd.slice.16000-16999.json',
    'mpd.slice.17000-17999.json',
    'mpd.slice.18000-18999.json',
    'mpd.slice.19000-19999.json',
    'mpd.slice.20000-20999.json',
    'mpd.slice.21000-21999.json',
    'mpd.slice.22000-22999.json',
    'mpd.slice.23000-23999.json',
    'mpd.slice.24000-24999.json',
    'mpd.slice.25000-25999.json',
    'mpd.slice.26000-26999.json',
    'mpd.slice.27000-27999.json',
    'mpd.slice.28000-28999.json',
    'mpd.slice.29000-29999.json',
    'mpd.slice.30000-30999.json',
]

all_data = []

for file_path in file_paths:
    with open(file_path, 'r') as file:
        data = json.load(file)
        for playlist in data['playlists']:
            playlist_name = playlist['name']
            for track in playlist['tracks']:
                all_data.append({
                    "playlist_name": playlist_name,
                    "position_in_playlist": track["pos"],
                    "track_id": track['track_uri'],
                    "track_name": track['track_name'],
                    "artist_name": track['artist_name'],
                    "duration_ms": track['duration_ms'],
                    "album_name": track['album_name']
                })
                # Add and remove for DBSCAN
                '''
                if len(all_data) >= 1000:
                    break
            if len(all_data) >= 1000:
                break
        if len(all_data) >= 1000:
            break
            '''
df = pd.DataFrame(all_data)

In [24]:
d1

,0
0,Lake Street Dive
1,Heartless Bastards
2,Vulfpeck
3,Alabama Shakes
4,Bahamas
...,...
150,Ezra Furman
151,Abby Sage
152,Alex Ebert
153,JosÃ© GonzÃ¡lez


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1077598 entries, 0 to 1077597
Data columns (total 7 columns):
 #   Column                Non-Null Count    Dtype 
---  ------                --------------    ----- 
 0   playlist_name         1077598 non-null  object
 1   position_in_playlist  1077598 non-null  int64 
 2   track_id              1077598 non-null  object
 3   track_name            1077598 non-null  object
 4   artist_name           1077598 non-null  object
 5   duration_ms           1077598 non-null  int64 
 6   album_name            1077598 non-null  object
dtypes: int64(2), object(5)
memory usage: 57.6+ MB


In [6]:
track_counts = df.groupby(['track_id', 'track_name', 'artist_name']).size().reset_index(name='count')

In [7]:
min_count = track_counts['count'].min()
max_count = track_counts['count'].max()
track_counts['popularity_score'] = (
    ((track_counts['count'] - min_count) / (max_count - min_count) * 9 + 1
).astype(int))

In [8]:
df_with_popularity = df.merge(
    track_counts[['track_id', 'popularity_score']],
    on='track_id',
    how='left'
)

# Popularity Score

## What Does the Score Tell You?
### High Popularity Score (8-10):

#### Tracks with high scores are included in many playlists, indicating they are widely popular or well-liked.

### Medium Popularity Score (4-7):

#### Tracks with medium scores are moderately popular and appear in a reasonable number of playlists.

### Low Popularity Score (1-3):

#### Tracks with low scores are less popular and appear in only a few playlists.

In [26]:
df_with_popularity.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1077598 entries, 0 to 1077597
Data columns (total 8 columns):
 #   Column                Non-Null Count    Dtype 
---  ------                --------------    ----- 
 0   playlist_name         1077598 non-null  object
 1   position_in_playlist  1077598 non-null  int64 
 2   track_id              1077598 non-null  object
 3   track_name            1077598 non-null  object
 4   artist_name           1077598 non-null  object
 5   duration_ms           1077598 non-null  int64 
 6   album_name            1077598 non-null  object
 7   popularity_score      1077598 non-null  int32 
dtypes: int32(1), int64(2), object(5)
memory usage: 69.9+ MB


In [9]:
df_with_popularity['popularity_score']

0          1
1          1
2          1
3          1
4          1
          ..
1077593    1
1077594    1
1077595    1
1077596    1
1077597    1
Name: popularity_score, Length: 1077598, dtype: int32

In [10]:
df_with_popularity[df_with_popularity['popularity_score'] == 6]

,playlist_name,position_in_playlist,track_id,track_name,artist_name,duration_ms,album_name,popularity_score
109,PB,1,spotify:track:2KpCpk6HjXXLb7nnXoXA5O,Ni**as In Paris,JAY Z,219320,Top Five,6
131,PB,23,spotify:track:12REd1n8PeiHHWOh066tpr,Never Be Like You,Flume,233337,Skin,6
352,hype,0,spotify:track:6gBFPUFcJLzWGx4lenP6h2,goosebumps,Travis Scott,243836,Birds In The Trap Sing McKnight,6
353,hype,1,spotify:track:6p8NuHm8uCGnn2Dtbtf7zE,Slippery (feat. Gucci Mane),Migos,304041,Culture,6
375,hype,23,spotify:track:7wGoVu4Dady5GV0Sv4UIsx,rockstar,Post Malone,218320,rockstar,6
...,...,...,...,...,...,...,...,...
1076619,rap,4,spotify:track:2FBUoWkIuQXwayw2RNo5l6,Hotline Bling,Drake,267066,Views,6
1076705,chill,80,spotify:track:1Tt4sE4pXi57mTD1GCzsqm,Needed Me,Rihanna,191600,ANTI,6
1076716,n/a,1,spotify:track:7mldq42yDuxiUNn08nvzHO,Body Like A Back Road,Sam Hunt,165386,Body Like A Back Road,6
1077013,Partay,66,spotify:track:7oK9VyNzrYvRFo7nQEYkWN,Mr. Brightside,The Killers,222586,Hot Fuss,6


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1077598 entries, 0 to 1077597
Data columns (total 7 columns):
 #   Column                Non-Null Count    Dtype 
---  ------                --------------    ----- 
 0   playlist_name         1077598 non-null  object
 1   position_in_playlist  1077598 non-null  int64 
 2   track_id              1077598 non-null  object
 3   track_name            1077598 non-null  object
 4   artist_name           1077598 non-null  object
 5   duration_ms           1077598 non-null  int64 
 6   album_name            1077598 non-null  object
dtypes: int64(2), object(5)
memory usage: 57.6+ MB


In [12]:
df.head()

,playlist_name,position_in_playlist,track_id,track_name,artist_name,duration_ms,album_name
0,Wedding Music,0,spotify:track:2rb4cO7RczQFSvpjTJ4C2P,Always Forever,Phil Wickham,281880,Phil Wickham
1,Wedding Music,1,spotify:track:017nSBNU2XHwMV0NCWZCqg,Divine Romance,Phil Wickham,298026,Phil Wickham
2,Wedding Music,2,spotify:track:0W5TB5VNs0J16suh3r67P1,Messiah / You're Beautiful,Phil Wickham,293720,Cannons
3,Wedding Music,3,spotify:track:1gBnG1MiTNBBVzmuwP7Wii,Love Is Not A Fight,Warren Barfield,222986,Worth Fighting For
4,Wedding Music,4,spotify:track:0b99xsUKkETGwZGzpX987r,When I Say I Do,Matthew West,246000,Hold You Up EP


# This is where I started my work, feel free to view but is not accurate for the 1000 rows for DBSCAN.

In [13]:
track_name = df.drop_duplicates(subset=['track_name'])

In [14]:
track_name.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 177749 entries, 0 to 1077595
Data columns (total 7 columns):
 #   Column                Non-Null Count   Dtype 
---  ------                --------------   ----- 
 0   playlist_name         177749 non-null  object
 1   position_in_playlist  177749 non-null  int64 
 2   track_id              177749 non-null  object
 3   track_name            177749 non-null  object
 4   artist_name           177749 non-null  object
 5   duration_ms           177749 non-null  int64 
 6   album_name            177749 non-null  object
dtypes: int64(2), object(5)
memory usage: 10.8+ MB


In [15]:
artists = df.drop_duplicates(subset=['artist_name'])

In [16]:
artists.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45454 entries, 0 to 1077561
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   playlist_name         45454 non-null  object
 1   position_in_playlist  45454 non-null  int64 
 2   track_id              45454 non-null  object
 3   track_name            45454 non-null  object
 4   artist_name           45454 non-null  object
 5   duration_ms           45454 non-null  int64 
 6   album_name            45454 non-null  object
dtypes: int64(2), object(5)
memory usage: 2.8+ MB


In [17]:
playlists = df.drop_duplicates(subset=['playlist_name'])

In [18]:
playlists.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8650 entries, 0 to 1077551
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   playlist_name         8650 non-null   object
 1   position_in_playlist  8650 non-null   int64 
 2   track_id              8650 non-null   object
 3   track_name            8650 non-null   object
 4   artist_name           8650 non-null   object
 5   duration_ms           8650 non-null   int64 
 6   album_name            8650 non-null   object
dtypes: int64(2), object(5)
memory usage: 540.6+ KB


In [19]:
'''
X = df_with_popularity[['duration_ms', 'popularity_score']]

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

eps = 0.8
min_samples = 50

dbscan = DBSCAN(eps=eps, min_samples=min_samples)
dbscan.fit(X_scaled)

df_with_popularity['cluster'] = dbscan.labels_

print(df_with_popularity.head())

n_clusters = len(set(dbscan.labels_)) - (1 if -1 in dbscan.labels_ else 0)
n_noise = list(dbscan.labels_).count(-1)

print(f"Number of clusters: {n_clusters}")
print(f"Number of noise points: {n_noise}")

print(df_with_popularity['cluster'].value_counts())

SyntaxError: EOF while scanning triple-quoted string literal (803627880.py, line 23)

In [ ]:
'''
X = df_with_popularity[['duration_ms', 'popularity_score']]

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

eps = 0.7
min_samples = 50

dbscan = DBSCAN(eps=eps, min_samples=min_samples)
dbscan.fit(X_scaled)

df_with_popularity['cluster'] = dbscan.labels_

print(df_with_popularity.head())

n_clusters = len(set(dbscan.labels_)) - (1 if -1 in dbscan.labels_ else 0)
n_noise = list(dbscan.labels_).count(-1)

print(f"Number of clusters: {n_clusters}")
print(f"Number of noise points: {n_noise}")

print(df_with_popularity['cluster'].value_counts())

In [ ]:
'''
X = df_with_popularity[['duration_ms', 'popularity_score']]

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

eps = 0.09
min_samples = 50

dbscan = DBSCAN(eps=eps, min_samples=min_samples)
dbscan.fit(X_scaled)

df_with_popularity['cluster'] = dbscan.labels_

print(df_with_popularity.head())

n_clusters = len(set(dbscan.labels_)) - (1 if -1 in dbscan.labels_ else 0)
n_noise = list(dbscan.labels_).count(-1)

print(f"Number of clusters: {n_clusters}")
print(f"Number of noise points: {n_noise}")

print(df_with_popularity['cluster'].value_counts())